In [1]:
import datetime as dt
import numpy as np
import pandas as pd
import pytz

from numba import jit
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_hdf('./data_backtest_0.hdf').sort_values(by=['close_dt_pst','tsec'])
df['price_chg'] = df['close_next'] - df['close']
df['price_diff'] = df['price'] - df['close']
df['pct_diff'] = df['price_diff']/df['close']
df['close_dt_pst'] = df['close_dt_pst'].apply(lambda x: np.datetime64(x))
df['next_dt'] = df['next_dt'].apply(lambda x: np.datetime64(x))

In [3]:
df.head()

,close_dt_pst,tsec,next_dt,close,price,close_next,price_chg,price_diff,pct_diff
0,2021-03-03 19:33:00,35.0,2021-03-03 19:35:00,1570.71,1574.890688,1574.48,3.77,4.180688,0.002662
1,2021-03-03 19:33:00,35.0,2021-03-03 19:35:00,1570.71,1572.839180,1574.48,3.77,2.129180,0.001356
2,2021-03-03 19:35:00,15.0,2021-03-03 19:37:00,1574.48,1576.943434,1572.93,-1.55,2.463434,0.001565
3,2021-03-03 19:36:00,0.0,2021-03-03 19:38:00,1571.98,1574.888450,1572.30,0.32,2.908450,0.001850
5,2021-03-03 19:39:00,0.0,2021-03-03 19:41:00,1569.15,1573.588614,1565.59,-3.56,4.438614,0.002829


In [4]:
pct_buy = -0.005231
pct_flat = -0.002140
pct_sell = 0.000091
df_trades = pd.DataFrame(columns=['pos','open_time','close_time','open_price','close_price',
                                  'price_max','price_min','type'])
k = 0
df_trades.loc[k,'pos'] = 0
df_trades.loc[k,'price_min'] = df.loc[0,'price']
df_trades.loc[k,'price_min'] = df.loc[0,'price']

# column = np_df[:,0])
np_df = df[['tsec', 'price', 'pct_diff']].to_numpy()
close_dts = df['close_dt_pst'].to_numpy()

In [5]:
print(df.columns)
df_trades

Index(['close_dt_pst', 'tsec', 'next_dt', 'close', 'price', 'close_next',
       'price_chg', 'price_diff', 'pct_diff'],
      dtype='object')


,pos,open_time,close_time,open_price,close_price,price_max,price_min,type
0,0,NaN,NaN,NaN,NaN,NaN,1574.89,NaN


In [6]:
@jit(nopython=True)
def run_bt(np_df, times, pct_buy, pct_flat, pct_sell, num_reps):
    '''
        End result is going to be 
        columns: position, open price, close price, type, open_pct_diff, close_pct_diff, open_tsecs, close_tsecs
    
    
        For the loop, open_long=0, flip_long=1, open_short=2, flip_short=3, close=4
    '''
    output = np.empty((num_reps, 8), np.float64)
    output[:] = np.nan
    output[0, 0] = 0
    open_time_index = np.empty(num_reps)
    close_time_index = np.empty(num_reps)
    last_trade = 0
    
    for i in range(num_reps):
        trade_type=None
        row = np_df[i, :]
        tt = times[i]
        pct_diff = row[-1]
        price = row[1]
        tsec = row[0]
        if pct_diff < pct_buy:
            if output[last_trade, 0] == 0:
                trade_type = 0
            elif output[last_trade, 0] == -1:
                trade_type = 1
        elif pct_diff > pct_sell:
            if output[last_trade, 0] == 0:
                trade_type = 2
            elif output[last_trade, 0] == 1:
                trade_type = 3

        elif output[last_trade, 0] == 1 and pct_diff > pct_flat:
            trade_type = 4
        elif output[last_trade, 0] == -1 and pct_diff < pct_flat:
            trade_type = 4

        if trade_type is not None:
            if trade_type == 0 or trade_type == 2:
                last_trade = i
                if trade_type == 0:
                    output[last_trade, 0] = 1
                    output[last_trade, 3] = trade_type
                else:
                    output[last_trade, 0] = -1
                    output[last_trade, 3] = trade_type
                
                output[last_trade, 1] = price
                output[last_trade, 4] = pct_diff
                output[last_trade, -2] = tsec
                open_time_index[last_trade] = last_trade

            else:
                #first close and then open a new pos if we are flipping
                output[last_trade, 0] = 0
                output[last_trade, 2] = price
                output[last_trade, 5] = pct_diff
                output[last_trade, -1] = tsec   
                close_time_index[last_trade] = i
                if trade_type == 1 or trade_type == 3:
                    last_trade = i
                    if trade_type == 1:
                        output[last_trade, 0] = 1
                    else:
                        output[last_trade, 0] = -1
                    output[last_trade, 1] = price
                    output[last_trade, 4] = pct_diff
                    output[last_trade, -2] = tsec
                    open_time_index[last_trade] = last_trade
                    if trade_type == 1:
                        output[last_trade, 3] = 0
                    else:
                        output[last_trade, 3] = 2

    return output, open_time_index, close_time_index

In [7]:
rez, open_time_index, close_time_index = run_bt(np_df, close_dts, pct_buy, pct_flat, pct_sell, len(np_df))
rez = pd.DataFrame(rez, columns=["position", "open_price", "close_price", 
                           "type", "open_pct_diff", "close_pct_diff", "open_tsecs", "close_tsecs"])

rez['open_time'] = open_time_index
rez['close_time'] = close_time_index
rez = rez.dropna()

rez['open_time'] = rez['open_time'].astype(int)
rez['close_time'] = rez['close_time'].astype(int)
rez['open_time'] = rez['open_time'].apply(lambda x: close_dts[x])
rez['close_time'] = rez['close_time'].apply(lambda x: close_dts[x])
rez['open_time'] += rez['open_tsecs'].apply(lambda x: dt.timedelta(seconds=int(x)))
rez['close_time'] += rez['close_tsecs'].apply(lambda x: dt.timedelta(seconds=int(x)))

In [8]:
def trade_type(x):
    if x == 0.:
        return "open_long"
    elif x == 1.:
        return "flip_long"
    elif x == 2.:
        return "open_short"
    elif x == 3.:
        return "flip_short"
    else:
        return "close"


rez["type"] = rez["type"].apply(lambda x: trade_type(x))
rez['pnl'] = 0
rez['edge'] = np.abs(rez['open_price'] - rez['open_price']/(1+rez['open_pct_diff']))
rez.loc[rez['type']=='open_short','pnl'] = rez['open_price'] - rez['close_price']
rez.loc[rez['type']=='open_long','pnl'] = -(rez['open_price'] - rez['close_price'])
rez.mean()

position             0.000000
open_price        2732.949476
close_price       2734.904321
open_pct_diff       -0.002846
close_pct_diff      -0.002463
open_tsecs          28.041991
close_tsecs         23.344671
pnl                  6.896974
edge                10.170260
dtype: float64

In [9]:
rez.sort_values(by='pnl')
rez

,position,open_price,close_price,type,open_pct_diff,close_pct_diff,open_tsecs,close_tsecs,open_time,close_time,pnl,edge
0,0.0,1574.890688,1548.770199,open_short,0.002662,-0.005247,35.0,35.0,2021-03-03 19:33:35,2021-03-03 19:53:35,26.120489,4.180688
44,0.0,1548.770199,1553.645889,open_long,-0.005247,-0.002116,35.0,35.0,2021-03-03 19:53:35,2021-03-03 19:53:35,4.875691,8.169801
46,0.0,1558.372660,1552.232777,open_short,0.000920,-0.003023,35.0,50.0,2021-03-03 19:53:35,2021-03-03 19:53:50,6.139883,1.432660
53,0.0,1556.908597,1552.691524,open_short,0.000841,-0.003081,25.0,30.0,2021-03-03 19:54:25,2021-03-03 20:21:30,4.217073,1.308597
104,0.0,1557.159490,1543.006741,open_short,0.002143,-0.002246,15.0,20.0,2021-03-03 20:23:15,2021-03-03 21:06:20,14.152748,3.329490
...,...,...,...,...,...,...,...,...,...,...,...,...
298395,0.0,2616.854734,2618.332069,open_long,-0.005871,-0.002122,20.0,15.0,2021-06-04 07:11:20,2021-06-04 07:12:15,1.477335,15.455266
298423,0.0,2601.617977,2609.509607,open_long,-0.005939,-0.001756,15.0,55.0,2021-06-04 07:16:15,2021-06-04 07:17:55,7.891630,15.542023
298445,0.0,2613.592308,2613.385716,open_long,-0.005331,-0.001354,35.0,5.0,2021-06-04 07:25:35,2021-06-04 07:26:05,-0.206591,14.007692
298453,0.0,2617.648961,2617.905412,open_short,0.000275,-0.002991,35.0,10.0,2021-06-04 07:26:35,2021-06-04 07:27:10,-0.256451,0.718961
